In [135]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import requests, re, json ,pprint
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import urllib.request
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from urllib.parse import urlparse
from time import sleep
import datetime
import pytz # new import
new_timezone = pytz.timezone("US/Eastern")


from dateparser.search import search_dates
import arrow
import datetime
from dateutil.parser import parse
import weasyprint 

In [166]:
driver_path='./chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')   
driver = webdriver.Chrome(driver_path,options=options)
# driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

In [167]:
# my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A'
my_url = 'https://www.barchart.com/futures/quotes/ZW*0/options?futuresOptionsView=split'
driver.get(my_url)
sleep(1)
# wa
# driver.close()

In [169]:
def reload_test_site():
    my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A&amp;lc=Ugw0nq07e-G5JyUsl094AaABAg'
    driver.get(my_url)
    sleep(1)

## Save Screenshot

In [ ]:
page = driver.page_source
weasyprint.HTML(file_obj=page).write_pdf('out.pdf')

In [50]:
def screenshot_page(webdriver,output_file='out.pdf'):
    page = webdriver.page_source
    weasyprint.HTML(file_obj=page).write_pdf(output_file)

## Create Datestamp

In [77]:
default_timezone=pytz.UTC
def get_capture_date():
    return datetime.datetime.now().strftime("%a %b %d, %Y")
def get_capture_time(timezone=pytz.UTC):
    return datetime.datetime.now(timezone).strftime("%H:%M:%S %Z")

In [79]:
get_capture_time()

'18:49:20 UTC'

## Extract Name

In [ ]:
stock_name = driver.find_element_by_class_name(name='symbol')

In [49]:
stock_name.text

"Wheat Mar '19"

## Extract Days to Expiration

In [98]:
def extract_days_to_expiration(webdriver):
    dt_expiration = webdriver.find_element_by_class_name(name='bc-options-toolbar__second-row')
    a = dt_expiration.find_element_by_xpath("//strong[contains(text(), 'Days')]")
    return a.text

In [97]:
dt_expiration = driver.find_element_by_class_name(name='bc-options-toolbar__second-row')
a = dt_expiration.find_element_by_xpath("//strong[contains(text(), 'Days')]")
a.text

'83 Days'

## Extract IV

In [102]:
iv = driver.find_element_by_class_name(name='bc-options-toolbar__second-row')
a = iv.find_element_by_xpath("//div[contains(text(), 'Implied Volatility')]/strong")
a.text

'21.01%'

In [103]:
def extract_iv(webdriver):
    iv = webdriver.find_element_by_class_name(name='bc-options-toolbar__second-row')
    a = iv.find_element_by_xpath("//div[contains(text(), 'Implied Volatility')]/strong")
    return a.text

## Extract Last

In [ ]:
iv = driver.find_element_by_class_name(name='pricechangerow')
iv.find_element_by_xpath("//span[@class='last-change']").text

In [ ]:
def extract_last(webdriver):
    iv = webdriver.find_element_by_class_name(name='pricechangerow')
    return iv.find_element_by_xpath("//span[@class='last-change']").text

### Extract Table

In [159]:
test = driver.find_elements_by_xpath("//td[@class='lastPrice_left']")


In [160]:
for item in test:
    print(item.text)

48-6s
44-6s
40-7s
37-3s
34-0s
30-7s
28-1s
25-4s
23-1s
21-0s
19-0s
17-1s
15-4s
14-0s
12-5s
11-3s
10-3s
9-3s
8-4s
7-6s
Last


## Extract Strike

In [162]:
test = driver.find_elements_by_xpath("//td[@class='strikePrice']")
for item in test:
    print(item.text)

470-0
475-0
480-0
485-0
490-0
495-0
500-0
505-0
510-0
515-0
520-0
525-0
530-0
535-0
540-0
545-0
550-0
555-0
560-0
565-0
Strike


## Extract Price Right

In [163]:
test = driver.find_elements_by_xpath("//td[@class='lastPrice_right']")
for item in test:
    print(item.text)

3-2s
4-1s
5-3s
6-6s
8-3s
10-2s
12-3s
14-6s
17-3s
20-2s
23-1s
26-3s
29-5s
33-1s
36-6s
40-4s
44-4s
48-4s
52-5s
56-6s
Last


## Extract Last and Strike

In [169]:
lp_left = driver.find_elements_by_xpath("//td[@class='lastPrice_left']")
strike_price = driver.find_elements_by_xpath("//td[@class='strikePrice']")
lp_right = driver.find_elements_by_xpath("//td[@class='lastPrice_right']")

for i in range(len(lp_left)):
    print(lp_left[i].text,strike_price[i].text,lp_right[i].text)


48-6s 470-0 3-2s
44-6s 475-0 4-1s
40-7s 480-0 5-3s
37-3s 485-0 6-6s
34-0s 490-0 8-3s
30-7s 495-0 10-2s
28-1s 500-0 12-3s
25-4s 505-0 14-6s
23-1s 510-0 17-3s
21-0s 515-0 20-2s
19-0s 520-0 23-1s
17-1s 525-0 26-3s
15-4s 530-0 29-5s
14-0s 535-0 33-1s
12-5s 540-0 36-6s
11-3s 545-0 40-4s
10-3s 550-0 44-4s
9-3s 555-0 48-4s
8-4s 560-0 52-5s
7-6s 565-0 56-6s
Last Strike Last
